In [1]:
# Calculate the band means and std for the RGBI kelp PA dataset

In [1]:
import numpy as np
import albumentations as A
from train.datamodule import DataModule 
from tqdm.auto import tqdm
import torch

NUM_BANDS = 4

# Transforms
transform = A.ToFloat(p=1.0)

# Load the dataset
dm = DataModule(
    data_dir="/home/taylor/data/KP-ACO-RGBI-Nov2023/",
    num_classes=2,
    batch_size=8,
    train_transforms=transform,
    tests_transforms=transform,
)
dm.setup()

In [2]:
DEVICE = torch.device("cuda")

In [3]:
# Calculate the mean and std for the dataset
x = torch.Tensor([0] * NUM_BANDS).to(DEVICE)
xsq = torch.Tensor([0] * NUM_BANDS).to(DEVICE)
count = 0

train_dataloader = dm.train_dataloader()

for img, label in tqdm(iter(train_dataloader), total=len(train_dataloader)):
    img = img.to(DEVICE)
    x += img.sum(dim=(0,1,2))
    xsq += torch.multiply(img,img).sum(dim=(0,1,2))
    count += img.shape[0] * img.shape[1] * img.shape[2]
    

In [4]:
# Calculate the mean and std
mean = x / count
std = torch.sqrt(xsq / count - torch.multiply(mean, mean))

print(f"Mean: {mean}")
print(f"Std: {std}")